In [1]:
from google.colab import drive
drive.mount("/content/mydrive")

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [2]:
%cd /content/mydrive/MyDrive/healthcare

/content/mydrive/MyDrive/healthcare


In [3]:
!nvidia-smi

Fri Nov 15 12:24:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   70C    P8              19W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install dependencies

In [4]:
!pip install gradio

In [5]:
!pip install transformers accelerate

In [6]:
import gradio as gr
import numpy as np

from transformers import AutoTokenizer
import transformers
import torch


## Load Fine-tuned Model from drive

In [7]:
global tokenizer, pipeline

model = "llama-2-7b-custom100-FineTuned"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Create the Chatbot function

In [15]:
# Define the function to generate bot responses
def chat_response(message):
    system_message = "Welcome! You're now communicating with an AI model trained to assist with information about general health disease. Feel free to ask about causes, symptoms, medications, and treatment options!"
    prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{message}[/INST]"

    # Generate a response with truncation enabled to avoid warnings
    sequences = pipeline(
        f'[INST] {prompt} [/INST]',
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=500,
        truncation=True  # Ensures inputs are truncated if too long
    )

    # Extract bot's message
    bot_message = ""
    for seq in sequences:
        bot_message = seq['generated_text'].replace(prompt, '').split('[/INST]')[-1].strip()

    # Return the bot's message in a format compatible with Gradio's chatbot component
    return message, bot_message

## Design Application UI using Gradio Interface

In [16]:
import gradio as gr
import random
import time

# Define the image path for doctor GPT
doctor_gpt_image_path = "/content/mydrive/MyDrive/healthcare/doctorGPT.png"

# Custom CSS for the appearance
custom_css = """
body {
    background-color: #fdf6e3; /* Soft pastel background color */
    font-family: Arial, sans-serif;
}
.sidebar {
    background-color: #aaacad; /* Darker color for sidebar */
    padding: 20px;
    border-radius: 10px;
}
.title {
    font-size: 1.8rem;
    font-weight: bold;
    color: #050505;
    text-align: center;
}
.description {
    font-size: 1rem;
    color: #050505;
    text-align: center;
}
.chatbox, .centered-input {
    width: 100%; /* Equal width for chatbox and input */
    max-width: 800px;
    margin: 0 auto;
    background-color: #ffffff;
    border: 1px solid #dddddd;
    border-radius: 10px;
    padding: 10px;
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
}
.centered-input {
    display: flex;
    justify-content: space-between;
    align-items: center;
    gap: 10px;
}
.clear-button {
    width: 120px;
    height: 40px;
    align-self: center;
}
.chatbot {
    height: 400px;
    overflow-y: auto;
}
"""

# Define Gradio interface with custom styling and chatbot functionality
with gr.Blocks(css=custom_css) as demo:
    with gr.Row():
        with gr.Column(scale=1, elem_classes="sidebar"):
            gr.Image(value=doctor_gpt_image_path, interactive=False, show_label=False, width=100)
            gr.Markdown("<h1 class='title'>Personal Health Assistant</h1>")
            gr.Markdown(
                """
                <p class='description'>
                Welcome! Feel free to ask about health-related questions, symptoms, and treatments.
                </p>
                """
            )
        with gr.Column(scale=3):
            with gr.Row():
                chatbot = gr.Chatbot(label="Chat with the AI Assistant", elem_classes="chatbox")
            with gr.Row(elem_classes="centered-input"):
                msg = gr.Textbox(placeholder="Type your question here...", label="Your Question", lines=1)
            with gr.Row(elem_classes="clear-button"):
                clear = gr.Button("Clear Chat", variant="secondary")

    # Respond function to update chat history
    def respond(message, chat_history):
        bot_message = chat_response(message)  # Get response from chat_response function
        chat_history.append(bot_message)  # Append both user and bot messages to history
        time.sleep(2)  # Simulate a small delay before returning the updated chat history
        return "", chat_history

    # Actions for buttons
    msg.submit(respond, [msg, chatbot], [msg, chatbot])  # Submit message
    clear.click(lambda: ([], []), inputs=[], outputs=[chatbot])  # Clear chat history




## Lunch Application

In [ ]:
demo.launch(share=True, debug=True)